In [20]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf
import glob
import numpy as np
import pandas as pd
import torch 

from transformers4rec import torch as tr
from merlin.io import Dataset

from merlin.core.dispatch import make_df 
from merlin.systems.dag import Ensemble  
from merlin.systems.dag.ops.pytorch import PredictPyTorch 
from merlin.systems.dag.ops.workflow import TransformWorkflow 

In [21]:
from merlin.schema import Schema
from merlin.io import Dataset

S3_PATH = "s3a://zarklab-token-recommender-ohio2/buy-sell-updated-2022-12-01-2023-11-30_repartition2/part-00000-7eb03832-5162-448f-b63b-72d216838392-c000.snappy.parquet"

dataset = Dataset(S3_PATH,engine='parquet')
schema = dataset.schema

In [22]:
import cloudpickle
MODEL_PATH = "/home/sagemaker-user/token_recommender/saved_model/"

loaded_model = cloudpickle.load(
                open(os.path.join(MODEL_PATH, "t4rec_model_class.pkl"), "rb")
            )

In [23]:
model = loaded_model.cuda()
model.eval()

Model(
  (heads): ModuleList(
    (0): Head(
      (body): SequentialBlock(
        (0): TabularSequenceFeatures(
          (to_merge): ModuleDict(
            (continuous_module): SequentialBlock(
              (0): ContinuousFeatures(
                (filter_features): FilterFeatures()
                (_aggregation): ConcatFeatures()
              )
              (1): SequentialBlock(
                (0): DenseBlock(
                  (0): Linear(in_features=3, out_features=64, bias=True)
                  (1): ReLU(inplace=True)
                )
              )
              (2): AsTabular()
            )
            (categorical_module): SequenceEmbeddingFeatures(
              (filter_features): FilterFeatures()
              (embedding_tables): ModuleDict(
                (buyAsset-list): Embedding(2765, 64, padding_idx=0)
                (token_category-list): Embedding(71, 64, padding_idx=0)
                (token_rank-list): Embedding(944, 64, padding_idx=0)
                (

In [24]:
import pandas as pd
from merlin.table import TensorTable, TorchColumn
from merlin.table.conversions import convert_col
import nvtabular as nvt

# df = cudf.read_parquet(train_paths, columns=model.input_schema.column_names)



In [25]:
WORKFLOW_PATH = "/home/sagemaker-user/token_recommender/preprocessWorkflow"

workflow = nvt.Workflow.load(WORKFLOW_PATH)

procesed = workflow.transform(dataset)


In [26]:
df = procesed.to_ddf().compute()

In [27]:
df.head()

,recipient,timestamp-last,buyAsset-list,txFee_eth_log_norm-list,buyQty1_log_norm-list,buyPrice_log_norm-list,token_category-list,token_rank-list,risky_flags-list
0,0x000000000085cd7bd617419ce5ff21c722ab2d38,1674525959,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2032, 4, ...","[0.045712452, -0.011456146, 0.17717513, -0.466...","[-0.97598433, -1.0480045, -1.0515182, -1.06182...","[0.60994124, -0.53573817, -0.70448, -3.3231096...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
1,0x000000000f4141775240685a4e005ed819846840,1675988951,"[5, 5, 5, 5, 3, 3, 3, 3, 153, 3, 3, 3, 3, 3]","[-0.5144607, -0.15142529, -0.34187222, -0.3648...","[-0.9072337, -0.906153, -0.90690935, -0.906970...","[-2.786001, -2.7822459, -2.784874, -2.7851288,...","[6, 6, 6, 6, 3, 3, 3, 3, 15, 3, 3, 3, 3, 3]","[5, 5, 5, 5, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]"
2,0x000000002d4db35b28c690a28a7b4d58f1e2c738,1695850991,"[4, 1426, 4, 4, 3, 4, 4, 4, 3, 4, 3, 4, 4, 181...","[-0.416033, -0.2832228, 0.029938223, 0.0102594...","[-1.0553724, -0.596958, -1.046597, -1.0505277,...","[-0.9581145, -0.4722736, -0.4668614, -0.639143...","[5, 7, 5, 5, 3, 5, 5, 5, 3, 5, 3, 5, 5, 14, 5,...","[3, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
3,0x000000006533ecb269f92a09d8541e84e485332f,1692740783,"[3, 3, 580, 5, 580, 580, 5]","[2.6783893, 0.47435308, -0.09856902, 0.3105012...","[-1.0434743, -0.96712285, 0.058398556, 0.18351...","[-0.29537424, 0.75593024, 0.32826248, 1.003739...","[3, 3, 26, 6, 26, 26, 6]","[4, 4, 3, 5, 3, 3, 5]","[3, 3, 3, 3, 3, 3, 3]"
4,0x00000000bc6d7378514b0bce5dee4d07d370ed1c,1671599507,"[3, 3]","[-0.7036802, -0.56444293]","[-1.0568126, -1.0532434]","[-1.241568, -0.97583354]","[3, 3]","[4, 4]","[3, 3]"


In [28]:
model.input_schema.column_names

['txFee_eth_log_norm-list',
 'buyQty1_log_norm-list',
 'buyPrice_log_norm-list',
 'buyAsset-list',
 'token_category-list',
 'token_rank-list',
 'risky_flags-list']

In [29]:
df_select = df[model.input_schema.column_names]

table = TensorTable.from_df(df_select.loc[:100])
for column in table.columns:
    table[column] = convert_col(table[column], TorchColumn)
model_input_dict = table.to_dict()

In [30]:
traced_model = torch.jit.trace(model, model_input_dict, strict=True)


In [31]:
input_schema = model.input_schema
output_schema = model.output_schema

In [32]:
input_schema

,name,tags,dtype,is_list,is_ragged,properties.value_count.min,properties.value_count.max,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension
0,txFee_eth_log_norm-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,True,2,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,buyQty1_log_norm-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,True,2,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,buyPrice_log_norm-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,True,2,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,buyAsset-list,"(Tags.ITEM, Tags.ID, Tags.CATEGORICAL, Tags.LIST)","DType(name='int64', element_type=<ElementType....",True,True,2,20,NaN,0.0,0.0,.//categories/unique.buyAsset.parquet,0.0,2764.0,buyAsset,2765.0,135.0
4,token_category-list,"(Tags.CATEGORICAL, Tags.LIST)","DType(name='int64', element_type=<ElementType....",True,True,2,20,NaN,0.0,0.0,.//categories/unique.token_category.parquet,0.0,70.0,token_category,71.0,17.0
5,token_rank-list,"(Tags.CATEGORICAL, Tags.LIST)","DType(name='int64', element_type=<ElementType....",True,True,2,20,NaN,0.0,0.0,.//categories/unique.token_rank.parquet,0.0,943.0,token_rank,944.0,74.0
6,risky_flags-list,"(Tags.CATEGORICAL, Tags.LIST)","DType(name='int64', element_type=<ElementType....",True,True,2,20,NaN,0.0,0.0,.//categories/unique.risky_flags.parquet,0.0,18.0,risky_flags,19.0,16.0


In [33]:
import shutil
ens_model_path = os.environ.get("ens_model_path", f"/home/sagemaker-user/token_recommender/models")
# Make sure we have a clean stats space for Dask
if os.path.isdir(ens_model_path):
    shutil.rmtree(ens_model_path)
os.mkdir(ens_model_path)

In [34]:
print(workflow.input_schema.column_names)


['buyAsset', 'token_category', 'token_rank', 'risky_flags', 'timestamp', 'txFee_eth', 'buyQty1', 'buyPrice', 'recipient']


In [35]:
torch_op = workflow.input_schema.column_names >> TransformWorkflow(workflow) >> PredictPyTorch(
    traced_model, input_schema, output_schema
)

ensemble = Ensemble(torch_op, workflow.input_schema)

In [36]:
ens_config, node_configs = ensemble.export(ens_model_path)
